In [ ]:
# Generic imports:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import re
import os

# Machine learning/Stats imports:
import keras
import tensorflow as tf
from sklearn.decomposition import PCA
from keras.utils.vis_utils import model_to_dot, plot_model

os.chdir('/home/matsen/Downloads/repos/vampire/')
import vampire
import vampire.xcr_vector_conversion as conversion
import vampire.tcr_vae as tcr_vae
import vampire.germline_cdr3_aa_tensor as aa_tensor
import vampire.models as models
import vampire.common as common
import vampire.preprocess_adaptive as preprocess

import importlib
importlib.reload(vampire)

def model_to_svg(model, path):
    with open(path, 'wb') as fp:
        fp.write(model_to_dot(model).create(prog='dot', format='svg'))

### Real data

In [ ]:
df = preprocess.apply_all_filters(
    preprocess.read_adaptive_tsv('vampire/pipe_main/sample_data/02-0249_TCRB.4000.tsv.bz2'))

print(df.iloc[range(1000, 1005), [1,2,3]].to_html(index=False))

### `basic` model _before training_

In [ ]:
d = tcr_vae.TCRVAE.default_params()
d['model'] = 'basic'
v = tcr_vae.TCRVAE(d)
model_to_svg(v.decoder, '/home/matsen/Downloads/basic_decoder.svg')
print(v.generate(5).to_html(index=False))

### `count_match` model _before training_

In [ ]:
d = tcr_vae.TCRVAE.default_params()
d['model'] = 'count_match'
v = tcr_vae.TCRVAE(d)
model_to_svg(v.decoder, '/home/matsen/Downloads/count_match_decoder.svg')
print(v.generate(5).to_html(index=False))

In [ ]:
plot_model(v.vae, show_shapes=True, to_file='/home/matsen/Downloads/model.png')

In [ ]:
d = tcr_vae.TCRVAE.default_params()
d['model'] = 'count_match'
v = tcr_vae.TCRVAE(d)

model_to_svg(v.vae, '/home/matsen/Downloads/count_match.svg')

In [ ]:
os.chdir('/home/matsen/Downloads/repos/vampire/vampire/_ignore/plotting/2019-01-18-deneuter-exclude-js')
v = tcr_vae.TCRVAE.of_directory('deneuter-2018-12-31.train/0.75/basic/')

test = pd.read_csv('H10_B0/H10_B0.for-test.csv',
        usecols=['amino_acid', 'v_gene', 'j_gene'])

test_onehot = conversion.unpadded_tcrbs_to_onehot(test, 30)

z_mean,_ = v.encode(test_onehot)

pca = PCA(n_components=2)
pca.fit(z_mean)

z_mean_pcs = pca.transform(z_mean)
test['pc_1'] = z_mean_pcs[:, 0]
test['pc_2'] = z_mean_pcs[:, 1]

test.to_csv('pcs.csv', index=False)